#  Kinesis Data Firehiseを利用したデータパイプラインの演習

## 0. 準備


S3に配置されたParquetファイルを Redshift Spectrumを通してクエリします

# 9-1. S3バケットを調べてしましょう

Kinesis Daata Firehose によりParquetに変換しS3へ保存されていることを確認しましょう

In [ ]:
!aws s3 ls  --recursive s3://{bucket_name}/{table_name}/

# 9-2. Redshiftに接続

必要な Python Package をインポートします。

In [ ]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import boto3
import json

## パラメータをCloudFormationのスタックから取得

AWS CloudFormation で入力したパラメータおよび出力を取得します。

In [ ]:
# Please edit stack name
stack_name = 'step1'


cfn = boto3.client('cloudformation')
response = cfn.describe_stacks(StackName=stack_name)['Stacks'][0]

for item in response['Parameters']:
    if item['ParameterKey'] == 'MasterUserName':
        db_user = item['ParameterValue']
    elif item['ParameterKey'] == 'DatabaseName':
        db_name = item['ParameterValue']

db_port='5439'
        
for item in response['Outputs']:
    if item['OutputKey'] == 'RedshiftClusterEndpoint':
        cluster_endpoint = item['OutputValue'].split(':')[0]
    elif item['OutputKey'] == 'RedshiftClusterName':
        cluster_name = item['OutputValue']
    elif item['OutputKey'] == 'RedshiftClusterRole':
        redshift_role = item['OutputValue']
        
#  show parameters
print('db_user: {}'.format(db_user))
print('db_name: {}'.format(db_name))
print('db_port: {}'.format(db_port))
print('cluster_endpoint: {}'.format(cluster_endpoint))
print('cluster_name: {}'.format(cluster_name))
print('redshift_role: {}'.format(redshift_role))


## Credential 取得

Amazon Redshift へアクセスするための、[一時的データベースユーザー認証情報]を取得します。
(https://docs.aws.amazon.com/ja_jp/redshift/latest/mgmt/generating-iam-credentials-cli-api.html)


In [ ]:
# get temporal cluster credentials
redshift = boto3.client('redshift')
credentials = redshift.get_cluster_credentials(
    DbUser=db_user, 
    DbName=db_name, 
    ClusterIdentifier=cluster_name, 
    DurationSeconds=3600,
    AutoCreate=False
)

tmp_db_user = credentials['DbUser']
tmp_db_password = credentials['DbPassword']

print('User: {}'.format(tmp_db_user ))
print('DB password: {}'.format( tmp_db_password))

## Redshift DB接続

Python 用の PostgreSQL ドライバである psycopg2 を利用して Redshift へアクセスします。

In [ ]:
# connect to Redshift
conn = psycopg2.connect(
    host=cluster_endpoint, 
    port=db_port, 
    dbname=db_name, 
    user=tmp_db_user, 
    password=tmp_db_password
)
conn.autocommit = True
print(conn)

# 9-3. Spectrumを通して S3を検索しましょう

外部テーブルを登録しましょう

In [ ]:
sql_create_table= [
"""
    CREATE EXTERNAL SCHEMA workshop2 from data catalog
    DATABASE 'kinesislab'
    IAM_ROLE  '{}'
    create external database if not exists;
"""
]

In [ ]:
with conn.cursor() as cur:
    for sql in sql_create_table:
        cur.execute(sql.format(redshift_role))
        print('Done: ', sql.format(redshift_role))

In [ ]:
sql = '''select * from svv_external_schemas;'''
%time pd.read_sql(sql=sql, con=conn)

In [ ]:
sql = '''select * from svv_external_tables'''
%time pd.read_sql(sql=sql, con=conn)

### nyctaxitripsテーブルのクエリ

In [ ]:
sql_query = '''select *
from
  workshop2.nyctaxitrips
limit
  10;
'''

%time pd.read_sql(sql=sql_query, con=conn)

In [ ]:
with conn.cursor() as cur:
    for sql in sql_create_table:
        cur.execute(sql.format(accountid, redshift_role))
        print('Done: ', sql)

### Parquetファイルの検索　（カウント）

In [ ]:
sql_query = '''select
 year,month,day,hour , count(1) as count
from
   workshop2.nyctaxitrips
group by
   year,month,day,hour 
'''

%time pd.read_sql(sql=sql_query, con=conn)

## Close処理

最後に、psycopg2 の connection を閉じます。

In [ ]:
conn.close()